In [ ]:
! pip install -qU llama-index llama-index-llms-anthropic llama-index-embeddings-bedrock 

In [10]:
import os
import getpass
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API key:")

Anthropic API key: ········


In [11]:
!mkdir -p '../samples/data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O '../samples/data/10k/uber_2021.pdf'

--2024-04-12 15:45:39--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘../samples/data/10k/uber_2021.pdf’

../samples/data/10k 100%[===================>]   1.79M  4.80MB/s    in 0.4s    

2024-04-12 15:45:40 (4.80 MB/s) - ‘../samples/data/10k/uber_2021.pdf’ saved [1880483/1880483]



In [16]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.anthropic import Anthropic

Settings.chunk_size = 300
Settings.chunk_overlap = 20

embed = BedrockEmbedding(
    region_name="us-west-2",
    profile_name="sandbox",
    model="cohere.embed-english-v3"
)

llm = Anthropic(model="claude-3-opus-20240229")

In [17]:
uber_docs = SimpleDirectoryReader(input_files=["../samples/data/10k/uber_2021.pdf"]).load_data()

In [18]:
uber_index = VectorStoreIndex.from_documents(uber_docs, embed_model=embed, show_progress=True)

Parsing nodes:   0%|          | 0/307 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1304 [00:00<?, ?it/s]

In [19]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

In [20]:
query_engine_tool = QueryEngineTool(
    query_engine=uber_engine,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    )
)

In [21]:
from llama_index.core.agent import FunctionCallingAgentWorker, AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [query_engine_tool],
    llm=llm,
    verbose=True
)

agent = AgentRunner(agent_worker)

In [22]:
response = agent.chat("Tell me both the risk factors and tailwinds for Uber?")
print(str(response))

Added user message to memory: Tell me both the risk factors and tailwinds for Uber?
=== LLM Response ===
<thinking>
The uber_10k tool provides information about Uber's financials for 2021, which likely includes details on risk factors and tailwinds the company faces. The input parameter requires a detailed plain text question. The user's request for both risk factors and tailwinds for Uber can be answered by this tool without needing any additional information from the user.
</thinking>
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the key risk factors and tailwinds mentioned for Uber in their 2021 10-K filing?"}
=== Function Output ===
Based on the excerpts from Uber's 2021 10-K filing, some of the key risk factors and challenges mentioned include:

- Difficulties in implementing and maintaining financial systems and processes to enable compliance across multiple offerings and jurisdictions
- Import/export restrictions and changes in trade regulati